# Loading and splitting Dataset
The goal of this notebook is to continue the development of the DeepTropism model using a Pytorch.<br>
The dataset was already created on the previous notebook and all the HIV-1 env V3 loop sequences where aligned.


In [1]:
# Libraries used on the analysis
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import os
import random

First we load the Dataframe with all the sequences published on referenced articles.<br>
The D


In [5]:
df = pd.read_csv('/home/gabriel/Documents/Repos/DeepTropism/datasets/processed_tsv/alldataset_full_curated.tsv', 
                 sep='\t')
df.head()

,seq_name,dataset,label,sequence,sequence_aligned,label_numeric
0,RAB014775,newdb,CCR5,CTRPSNNTRTGITIGPGQVWYRTGDIIGDIRKAYC,CTRPSNNT-RTGI---T--I--G-P--G--Q-VWY---R-T--GDI...,0
1,RAB014776,newdb,CCR5,CTRPSNNTRTSITIGPGQVWYRTGDIIGDIRQAYC,CTRPSNNT-RTSI---T--I--G-P--G--Q-VWY---R-T--GDI...,0
2,RAB014778,newdb,CCR5,CTRPSNNTRTSITIGPGQVWYRTGDIIGDIRKAYC,CTRPSNNT-RTSI---T--I--G-P--G--Q-VWY---R-T--GDI...,0
3,RAB014781,newdb,CCR5,CTRPSNNTRTSVTIGPGQVWYRTGDIIGDIRQAYC,CTRPSNNT-RTSV---T--I--G-P--G--Q-VWY---R-T--GDI...,0
4,RAB014834,newdb,CCR5,CTRPSNNTRTSITIGPGQVWYRTGDIIGNIRKAYC,CTRPSNNT-RTSI---T--I--G-P--G--Q-VWY---R-T--GDI...,0


In [6]:
### Check the size of the column of the alig

In [7]:
set(df['sequence_aligned'].apply(len))

{60}

In [5]:
df = df[df.label != 'validation']
df.shape

(9550, 6)

In [6]:
df.label.value_counts()

CCR5     7705
CXCR4     937
R5X4      908
Name: label, dtype: int64

In [7]:
df.label_numeric.value_counts()

0    7705
1    1845
Name: label_numeric, dtype: int64

# Filtering the dataset by unique sequences
To improve the quality of our trainning and avoid bias we are going to create a dataset with only unique sequences from the original Dataframe.
This is going to be the dataset for the development of our model based on Deep Neural Network.

In [8]:
df_unique = df.drop_duplicates(subset=['sequence_aligned'], keep='first')

In [9]:
df_unique.head()

,seq_name,dataset,label,sequence,sequence_aligned,label_numeric
0,RAB014775,newdb,CCR5,CTRPSNNTRTGITIGPGQVWYRTGDIIGDIRKAYC,CTRPSNNT-RTGI---T--I--G-P--G--Q-VWY---R-T--GDI...,0
1,RAB014776,newdb,CCR5,CTRPSNNTRTSITIGPGQVWYRTGDIIGDIRQAYC,CTRPSNNT-RTSI---T--I--G-P--G--Q-VWY---R-T--GDI...,0
2,RAB014778,newdb,CCR5,CTRPSNNTRTSITIGPGQVWYRTGDIIGDIRKAYC,CTRPSNNT-RTSI---T--I--G-P--G--Q-VWY---R-T--GDI...,0
3,RAB014781,newdb,CCR5,CTRPSNNTRTSVTIGPGQVWYRTGDIIGDIRQAYC,CTRPSNNT-RTSV---T--I--G-P--G--Q-VWY---R-T--GDI...,0
4,RAB014834,newdb,CCR5,CTRPSNNTRTSITIGPGQVWYRTGDIIGNIRKAYC,CTRPSNNT-RTSI---T--I--G-P--G--Q-VWY---R-T--GDI...,0


## Creating Dataframes for comparing performance against other methods
In order to evaluate our model against the others already published, we are going to create separate Dataframes for each method filterinf by the 'dataset' column

In [10]:
df.dataset.value_counts()

newdb         2998
cm            2679
hivcopred     2335
geno2pheno    1188
webpssm        350
Name: dataset, dtype: int64

In [11]:
df_newdb = df[df.dataset == 'newdb']
df_cm = df[df.dataset == 'cm']
df_hivcopred = df[df.dataset == 'hivcopred']
df_geno2pheno = df[df.dataset == 'geno2pheno']
df_webpssm = df[df.dataset == 'webpssm']

# Spliting the Dataset for Cross Validation
We are going to create indices and set it to variables to make our cross validation reproducible. Our dataset is going to consist on:<br>
* Trainning = 80 %
* Validation = 10 %
* Test = 10 %

In [12]:
# Create a list of indices and shuffle it using seed
random.seed(42)
size = df_unique.shape[0]
list_indices = list(range(size))
random.shuffle(list_indices)

In [13]:
# Now create the list of indices for trainning, validation and test
test_indices = list_indices[:int(size/10)]
train_val_indices = list_indices[int(size/10):]

In [14]:
len(test_indices)

360

In [15]:
len(train_val_indices)

3248

In [16]:
assert len(train_val_indices) + len(test_indices) == len(list_indices), "Splitting indices with error"

## Creating the Dataloaders for trainning

In [17]:
def get_array_from_sequence(protein_sequence):
    """
    Function to convert a protein sequence into a tensor.
    Each amino acid is represented by an numpy array of zeros of size 26,
    and the dict_aa_pos defines the position to be converted to 1.
    
    The function iterates over the protein sequences and stacks the arrays.
    At the end the arrays are linearized and converted to a tensor of size
    n x 26, with n the size of the protein.
    
    If the character is not present on the dict_aa_pos (eg. '-') the respective
    array is formed by zeros, and represents a missing value.
    """
    dict_aa_pos = {
    'A':1, 'R':2, 'N':3, 'D':4, 'C':5, 'Q':6, 'E':7, 'G':8,
    'H':9, 'I':10, 'L':11, 'K':12, 'M':13, 'F':14, 'P':15, 
    'O':16, 'S':17, 'U':18, 'T':19, 'W':20, 'Y':21, 'V':22, 
    'B':23, 'Z':24, 'J':25, 'X':0}
    
    f_array = np.zeros(26)
    for aa in protein_sequence:
        arr = np.zeros(26)
        if dict_aa_pos.get(aa):
            arr[dict_aa_pos.get(aa)] = 1
        f_array = np.vstack((f_array, arr))
    f_array = np.delete(f_array, 0,0)
    
    #return torch.from_numpy((f_array.flatten()).astype(float))
    return f_array.flatten().astype(float)

In [18]:
# Create list to append data from the df
list_data = []
list_labels = []

# Convert the sequences and labels to arrays to use as data on pytorch
for index, row in df.iterrows():
    list_data.append(get_array_from_sequence(str(row.sequence_aligned)))
    list_labels.append(int(row.label_numeric))

In [19]:
len(list_data) == len(list_labels)

True

In [20]:
# For Test set
test_data = []
test_label = []
for j in test_indices:
    test_data.append(list_data[j])
    test_label.append(np.array(list_labels[j]))

test_tensor_x = torch.stack([torch.from_numpy(i) for i in test_data]) # transform to torch tensors
test_tensor_y = torch.stack([torch.from_numpy(i) for i in test_label])

test_dataset = torch.utils.data.TensorDataset(test_tensor_x,test_tensor_y) # create your test dataset
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=64) # create your dataloader


In [21]:
# Now we define the
len(train_val_indices)

3248

In [22]:
crossval_val_indices = np.array_split(train_val_indices,5)

In [23]:
len(crossval_val_indices[0])

650

In [24]:
# For Trainning and validation set
# Define the cross validation indices for trainning and validation sets
crossval_val_indices = np.array_split(train_val_indices,5)

# Iterate over crossval_val_indices defining the Dataloaders
for n in range(len(crossval_val_indices)):
    print(f'Cross Validation: {n + 1}')
    trainning_data = []
    trainning_label = []
    validation_data = []
    validation_label = []

    validation_indices = list(crossval_val_indices[n])
    trainning_indices = list(set(train_val_indices) - set(validation_indices))

    for j in validation_indices:
        validation_data.append(list_data[j])
        validation_label.append(np.array(list_labels[j]))

    validation_tensor_x = torch.stack([torch.from_numpy(i) for i in validation_data]) # transform to torch tensors
    validation_tensor_y = torch.stack([torch.from_numpy(i) for i in validation_label])

    validation_dataset = torch.utils.data.TensorDataset(validation_tensor_x,validation_tensor_y) # create your test dataset
    validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=64) # create your dataloader
    
    for k in trainning_indices:
        trainning_data.append(list_data[k])
        trainning_label.append(np.array(list_labels[k]))

    trainning_tensor_x = torch.stack([torch.from_numpy(i) for i in trainning_data]) # transform to torch tensors
    trainning_tensor_y = torch.stack([torch.from_numpy(i) for i in trainning_label])

    trainning_dataset = torch.utils.data.TensorDataset(trainning_tensor_x,trainning_tensor_y) # create your test dataset
    trainning_dataloader = torch.utils.data.DataLoader(trainning_dataset, batch_size=64) # create your dataloader

Cross Validation: 1
Cross Validation: 2
Cross Validation: 3
Cross Validation: 4
Cross Validation: 5


## Define the Deep Neural Network Architecture


In [120]:
class DeepTropism_1(nn.Module):
    def __init__(self):
        super(DeepTropism_1, self).__init__()
        self.linear1 = nn.Linear(1560,250)
        self.linear2 = nn.Linear(250,100)
        self.linear3 = nn.Linear(100,2)
    
    def forward(self,X):
        X = F.relu(self.linear1(X))
        X = F.relu(self.linear2(X))
        X = self.linear3(X)
        return F.log_softmax(X, dim=1)
 
model = DeepTropism_1().float()
#model = model.float()
model

DeepTropism_1(
  (linear1): Linear(in_features=1560, out_features=250, bias=True)
  (linear2): Linear(in_features=250, out_features=100, bias=True)
  (linear3): Linear(in_features=100, out_features=2, bias=True)
)

In [28]:
class DeepTropism_2(nn.Module):
    def __init__(self):
        super(DeepTropism_2, self).__init__()
        self.linear1 = nn.Linear(1560,750)
        self.linear2 = nn.Linear(750,250)
        self.linear3 = nn.Linear(250,100)
        self.linear4 = nn.Linear(100,2)
            
    def forward(self,X):
        X = F.relu(self.linear1(X))
        X = F.relu(self.linear2(X))
        X = F.relu(self.linear3(X))
        X = self.linear4(X)
        return F.log_softmax(X, dim=1)
 
model_2 = DeepTropism_2().float()
#model = model.float()
model_2

DeepTropism_2(
  (linear1): Linear(in_features=1560, out_features=750, bias=True)
  (linear2): Linear(in_features=750, out_features=250, bias=True)
  (linear3): Linear(in_features=250, out_features=100, bias=True)
  (linear4): Linear(in_features=100, out_features=2, bias=True)
)

In [26]:
# Define the Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# For Trainning and validation set DeepTropism_1
# Define the cross validation indices for trainning and validation sets
crossval_val_indices = np.array_split(train_val_indices,5)

# Iterate over crossval_val_indices defining the Dataloaders
for n in range(len(crossval_val_indices)):
    print(f'Cross Validation: {n + 1}')
    trainning_data = []
    trainning_label = []
    validation_data = []
    validation_label = []

    validation_indices = list(crossval_val_indices[n])
    trainning_indices = list(set(train_val_indices) - set(validation_indices))

    for j in validation_indices:
        validation_data.append(list_data[j])
        validation_label.append(np.array(list_labels[j]))

    validation_tensor_x = torch.stack([torch.from_numpy(i) for i in validation_data]) # transform to torch tensors
    validation_tensor_y = torch.stack([torch.from_numpy(i) for i in validation_label])

    validation_dataset = torch.utils.data.TensorDataset(validation_tensor_x,validation_tensor_y) # create your test dataset
    validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=64) # create your dataloader
    
    for k in trainning_indices:
        trainning_data.append(list_data[k])
        trainning_label.append(np.array(list_labels[k]))

    trainning_tensor_x = torch.stack([torch.from_numpy(i) for i in trainning_data]) # transform to torch tensors
    trainning_tensor_y = torch.stack([torch.from_numpy(i) for i in trainning_label])

    trainning_dataset = torch.utils.data.TensorDataset(trainning_tensor_x,trainning_tensor_y) # create your test dataset
    trainning_dataloader = torch.utils.data.DataLoader(trainning_dataset, batch_size=64) # create your dataloader
    
    # Instantiante new model
    #model = DeepTropism_1().float()
        
    # Define Cross Validation Trainning Loop
    for epoch in range(400):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainning_dataloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs.float())
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            #print(running_loss)
            #if i % 3239 == 0:    # print every 3239 mini-batches
            if epoch % 20 == 0 and i % 3239 == 0:
                print('[%d, %3d] loss: %.9f' %
                      (epoch + 1, i + 1, running_loss / 50))
                correct = 0
                total = 0
                error = 0
                labels_array = np.empty([0])
                predict_array = np.empty([0])

                with torch.no_grad():
                    for data in validation_dataloader:
                        images, labels = data
                        outputs = model(images.float())
                        _, predicted = torch.max(outputs.data, 1)

                        labels_array = np.concatenate([labels_array, labels])
                        predict_array = np.concatenate([predict_array, predicted])

                        total += labels.size(0)
                        correct += (predicted == labels).sum().item()
                        error += (predicted != labels).sum().item()

                print(f'Neural Network accuracy for validation set {n + 1}: {round(100.0 * correct/total, 2)}%')
            running_loss = 0.0
             

    print('Finished Training')
    
    correct = 0
    total = 0
    error = 0
    labels_array = np.empty([0])
    predict_array = np.empty([0])

    with torch.no_grad():
        for data in test_dataloader:
            images, labels = data
            outputs = model(images.float())
            _, predicted = torch.max(outputs.data, 1)

            labels_array = np.concatenate([labels_array, labels])
            predict_array = np.concatenate([predict_array, predicted])

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            error += (predicted != labels).sum().item()

    print(f'Neural Network accuracy on test set: {round(100.0 * correct/total, 2)}%')
    
    torch.save(model.state_dict(), f'model_cv{n+1}.ptb')

Cross Validation: 1
[1,   1] loss: 0.014180363
Neural Network accuracy for validation set 1: 19.69%
[21,   1] loss: 0.004136881
Neural Network accuracy for validation set 1: 80.31%
[41,   1] loss: 0.003257296
Neural Network accuracy for validation set 1: 80.46%
[61,   1] loss: 0.002938653
Neural Network accuracy for validation set 1: 88.46%
[81,   1] loss: 0.002780582
Neural Network accuracy for validation set 1: 88.77%
[101,   1] loss: 0.002657055
Neural Network accuracy for validation set 1: 90.15%
[121,   1] loss: 0.002561374
Neural Network accuracy for validation set 1: 90.62%


In [100]:
correct = 0
total = 0
error = 0
labels_array = np.empty([0])
predict_array = np.empty([0])

with torch.no_grad():
    for data in test_dataloader:
        images, labels = data
        outputs = model(images.float())
        _, predicted = torch.max(outputs.data, 1)
        
        labels_array = np.concatenate([labels_array, labels])
        predict_array = np.concatenate([predict_array, predicted])
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        error += (predicted != labels).sum().item()

print(f'Neural Network accuracy for test set: {round(100.0 * correct/total, 2)}%')

Neural Network accuracy for test set: 19.72%


In [27]:
def show_metrics(y_true, y_score):
    # True positive
    tp = np.sum(y_true * y_score)
    # False positive
    fp = np.sum((y_true == 0) * y_score)
    # True negative
    tn = np.sum((y_true==0) * (y_score==0))
    # False negative
    fn = np.sum(y_true * (y_score==0))

    # True positive rate (sensitivity or recall)
    tpr = tp / (tp + fn)
    # False positive rate (fall-out)
    fpr = fp / (fp + tn)
    # Precision
    precision = tp / (tp + fp)
    # True negatvie tate (specificity)
    tnr = 1 - fpr
    # F1 score
    f1 = 2*tp / (2*tp + fp + fn)
    # ROC-AUC for binary classification
    auc = (tpr+tnr) / 2
    # MCC
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    print("True positive: ", tp)
    print("False positive: ", fp)
    print("True negative: ", tn)
    print("False negative: ", fn)

    print("True positive rate (recall): ", tpr)
    print("False positive rate: ", fpr)
    print("Precision: ", precision)
    print("True negative rate: ", tnr)
    print("F1: ", f1)
    print("ROC-AUC: ", auc)
    print("MCC: ", mcc)

In [31]:
show_metrics(labels_array, predict_array)

True positive:  60.0
False positive:  11.0
True negative:  264
False negative:  22.0
True positive rate (recall):  0.7317073170731707
False positive rate:  0.04
Precision:  0.8450704225352113
True negative rate:  0.96
F1:  0.7843137254901961
ROC-AUC:  0.8458536585365853
MCC:  0.7289260178853867


In [29]:
# For Trainning and validation set DeepTropism_2
# Define the cross validation indices for trainning and validation sets
crossval_val_indices = np.array_split(train_val_indices,5)

# Iterate over crossval_val_indices defining the Dataloaders
for n in range(len(crossval_val_indices)):
    print(f'Cross Validation: {n + 1}')
    trainning_data = []
    trainning_label = []
    validation_data = []
    validation_label = []

    validation_indices = list(crossval_val_indices[n])
    trainning_indices = list(set(train_val_indices) - set(validation_indices))

    for j in validation_indices:
        validation_data.append(list_data[j])
        validation_label.append(np.array(list_labels[j]))

    validation_tensor_x = torch.stack([torch.from_numpy(i) for i in validation_data]) # transform to torch tensors
    validation_tensor_y = torch.stack([torch.from_numpy(i) for i in validation_label])

    validation_dataset = torch.utils.data.TensorDataset(validation_tensor_x,validation_tensor_y) # create your test dataset
    validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=64) # create your dataloader
    
    for k in trainning_indices:
        trainning_data.append(list_data[k])
        trainning_label.append(np.array(list_labels[k]))

    trainning_tensor_x = torch.stack([torch.from_numpy(i) for i in trainning_data]) # transform to torch tensors
    trainning_tensor_y = torch.stack([torch.from_numpy(i) for i in trainning_label])

    trainning_dataset = torch.utils.data.TensorDataset(trainning_tensor_x,trainning_tensor_y) # create your test dataset
    trainning_dataloader = torch.utils.data.DataLoader(trainning_dataset, batch_size=64) # create your dataloader
    
    # Instantiante new model
    #model = DeepTropism_1().float()
        
    # Define Cross Validation Trainning Loop
    for epoch in range(600):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainning_dataloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model_2(inputs.float())
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            #print(running_loss)
            #if i % 3239 == 0:    # print every 3239 mini-batches
            if epoch % 20 == 0 and i % 3239 == 0:
                print('[%d, %3d] loss: %.9f' %
                      (epoch + 1, i + 1, running_loss / 50))
                correct = 0
                total = 0
                error = 0
                labels_array = np.empty([0])
                predict_array = np.empty([0])

                with torch.no_grad():
                    for data in validation_dataloader:
                        images, labels = data
                        outputs = model_2(images.float())
                        _, predicted = torch.max(outputs.data, 1)

                        labels_array = np.concatenate([labels_array, labels])
                        predict_array = np.concatenate([predict_array, predicted])

                        total += labels.size(0)
                        correct += (predicted == labels).sum().item()
                        error += (predicted != labels).sum().item()

                print(f'Neural Network accuracy for validation set {n + 1}: {round(100.0 * correct/total, 2)}%')
            running_loss = 0.0
    

    
    
    correct = 0
    total = 0
    error = 0
    labels_array = np.empty([0])
    predict_array = np.empty([0])

    with torch.no_grad():
        for data in test_dataloader:
            images, labels = data
            outputs = model_2(images.float())
            _, predicted = torch.max(outputs.data, 1)

            labels_array = np.concatenate([labels_array, labels])
            predict_array = np.concatenate([predict_array, predicted])

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            error += (predicted != labels).sum().item()

    print(f'Neural Network accuracy on test set: {round(100.0 * correct/total, 2)}%')
    show_metrics(labels_array, predict_array)
print('Finished Training')

Cross Validation: 1
[1,   1] loss: 0.013994601
Neural Network accuracy for validation set 1: 19.69%
[21,   1] loss: 0.013994601
Neural Network accuracy for validation set 1: 19.69%
[41,   1] loss: 0.013994601
Neural Network accuracy for validation set 1: 19.69%


KeyboardInterrupt: 